**Курс** Специализация Data Science

**Дипломный проект.** Бриф учебного кейса 

«Модель прогнозирования стоимости жилья для агентства недвижимости»

**Цель**: разработать модель, которая позволила бы агентству недвижимости обойти конкурентов по скорости и качеству совершения
сделок.

**Описание данных**:
- 'status' — статус продажи;
- 'private pool' и 'PrivatePool' — наличие собственного бассейна;
- 'propertyType' — тип объекта недвижимости;
- 'street' — адрес объекта;
- 'baths' — количество ванных комнат;
- 'homeFacts' — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);
- 'fireplace' — наличие камина;
- 'city' — город;
- 'schools' — сведения о школах в районе;
- 'sqft' — площадь в футах;
- 'zipcode' — почтовый индекс;
- 'beds' — количество спален;
- 'state' — штат;
- 'stories' — количество этажей;
- 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система мультилистинга);
- 'target' — цена объекта недвижимости (целевой признак, который необходимо спрогнозировать)

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# зафиусируем RANDOM_SEED
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов
!pip freeze > requirements.txt

1) Рассмотрим и познакомимся с данными.


In [14]:
df = pd.read_csv('data/data.csv')
display(df.head())
df.info()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

Сразу удалим дубликаты.

In [15]:
count_rows = df.shape[0]
print(f'Количество строк в датасете: {count_rows}')

Количество строк в датасете: 377185


In [16]:
#удаляем дубликаты
df = df.drop_duplicates(ignore_index=True)

#считаем сколько строк было удалено
duplicate_rows = count_rows - df.shape[0]
print(f'Удалено дубликатов: {duplicate_rows}')

Удалено дубликатов: 50


Посмотрим на пропуски.

In [17]:
df.isnull().mean() * 100

status          10.584274
private pool    98.891378
propertyType     9.209699
street           0.000530
baths           28.188315
homeFacts        0.000000
fireplace       72.659127
city             0.009015
schools          0.000000
sqft            10.752118
zipcode          0.000000
beds            24.196640
state            0.000000
stories         39.952007
mls-id          93.386453
PrivatePool     89.311520
MlsId           17.730786
target           0.657589
dtype: float64

Удалим признаки 'mls-id' и 'MlsId', так как эти признаки нужны для идентификации объектов недвижимости для риэлторов, нам они не нужны.

In [18]:
df = df.drop(['MlsId', 'mls-id'], axis=1)

Признаки private pool и PrivatePool содержат дублирующую информацию и самое большое количество пропусков. Объединим их; похоже все эти пропуски свидетельствует об отсутствии бассейна.

In [19]:
print(df['private pool'].unique())
print(df['PrivatePool'].unique())

# создадим третий столбец и заполним его данными из двух, меняя данные на булевые переменные
df['private_pool_final'] = df['private pool'].fillna('') + df['PrivatePool'].fillna('')
df['private_pool_final'] = df['private_pool_final'].replace(['yes', 'Yes'], True)
df['private_pool_final'] = df['private_pool_final'].replace('', False)
# смотрим уникальные значения в новом столбце
df['private_pool_final'].unique()
# удаляем обработанные столбцы
df = df.drop(['private pool', 'PrivatePool'], axis=1)

[nan 'Yes']
[nan 'yes' 'Yes']


Обратим внимание на признак fireplace, который также содержит большое количество пропусков и удалим его.

In [20]:
df = df.drop(['fireplace'], axis=1)

Что касается признака stories, то количество пропусков тоже велико, а также особой информативности этот признак не несет, поэтому удалим его.

In [21]:
df = df.drop(['stories'], axis=1)

Рассмотрим признак sqft на наличие пропусков.

In [22]:
sqft_isnulldata = df['sqft'].isnull().mean()*100
print(f'Количество пропущенных значений: {sqft_isnulldata:.2f} %')
df['sqft'].unique()[0:20]

Количество пропущенных значений: 10.75 %


array(['2900', '1,947 sqft', '3,000 sqft', '6,457 sqft', nan, '897 sqft',
       '1,507', '3588', '1,930', '1,300 sqft', '3,130', '2,839 sqft',
       'Total interior livable area: 1,820 sqft', '2,454', '2,203',
       '3,325', '3,080 sqft', '1,612 sqft', '1,731 sqft',
       'Total interior livable area: 5,266 sqft'], dtype=object)

In [23]:
# оставим только числовые значения 
df['sqft'] = df['sqft'].str.replace(',', '').str.extract('(\d+)')

# смотрим количество пропусков в столбце
sqft_dataisnull = df['sqft'].isnull().sum()
print(f'Количество пропущенных значений sqft: {sqft_dataisnull}')

Количество пропущенных значений sqft: 41370


Рассмотрим признак beds и перенесем данные по площади в другие столбцы, они нам могут понадобиться для заполнения пропусков в столбце sqft

In [24]:
df['sqft_from_beds'] = df['beds'].str.extract(r'(\d+,\d+)\s+sqft', expand=False)
df['acres_from_beds'] = df['beds'].str.extract(r'([\d.]+)\s+acres', expand=False)

In [25]:
# удалим данные
df['beds'] = df['beds'].str.replace(r'(\d+,\d+)\s+sqft', '', regex=True)
df['beds'] = df['beds'].str.replace(r'([\d.]+)\s+acres', '', regex=True)

# изменим некоторые значения
df['beds'] = df['beds'].replace(['1 Bath, 2 Bedrooms, Cable TV Available, Dining Room, Eat-In Kitchen, Living Room', '1 Bath, 2 Bedrooms, Living Room, Range/Oven, Refrigerator', '1 Bath, 2 Bedrooms', '1 Bath, 2 Bedrooms, Eat-In Kitchen, Living Room, Range/Oven, Refrigerator'], 2)
df['beds'] = df['beds'].replace(['1 Bath, 3 or More Bedrooms, Cable TV Available, Dining Room, Eat-In Kitchen, Living Room, Range/Oven, Refrigerator', '3 or More Bedrooms, Dining Room, Living Room, Range/Oven, Refrigerator', '3 or More Bedrooms', '2 Baths, 3 or More Bedrooms'], 3)
df['beds'] = df['beds'].replace(["Based on Redfin's St Johns data, we estimate the home's value is $360,731, which is 2.2% less than its current list price.", "Based on Redfin's Raleigh data, we estimate the home's value is $708,248, which is 1.2% more than its current list price.", '-- bd', '-- sqft', '4 sqft', '60 sqft', '1 acre', '840 sqft', '540 sqft', '100 sqft', '871 sqft', '640 sqft', '831 sqft', '448 sqft', '248 sqft', '# Bedrooms 1st Floor'], 0)

# переносим данные по спальням в другие столбцы
df['beds_final1'] = df['beds'].str.extract(r'(\d+)\s*(?:Beds|bd)', expand=False)
df['beds_final2'] = df['beds'].str.extract(r'(\d+\.\d+|\d+)')

# создадим столбец beds_final и заполняем его данными из двух, заменяя пропуски на 0
df['beds_final'] = df['beds_final1'].fillna(0)
df['beds_final'] = df['beds_final2'].fillna(0)

# проверим на пропуски
beds_nulldata = df['beds_final'].isnull().mean()*100
print(f'Пропуски в столбце beds_final: {beds_nulldata} %')

# изменим тип данных
df['beds_final'] = df['beds_final'].astype(float)

# удалим отработанные столбцы
df = df.drop(['beds_final1', 'beds_final2', 'beds'], axis=1)

Пропуски в столбце beds_final: 0.0 %


Теперь заполним пропуски в столбце с площадью данными из столбца beds (sqft_from_beds и acres_from_beds)

In [26]:
# удалим лишние знаки
df['sqft_from_beds'] = df['sqft_from_beds'].replace(',', '', regex=True)

# заполним столбец данными из sqft_from_beds
df['sqft'] = df['sqft'].fillna(df['sqft_from_beds'])

# снова проверим на наличие пропусков
sqft_dataisnull_1 = df['sqft'].isnull().sum()
print(f'Количество пропущенных значений sqft после дополнения: {sqft_dataisnull_1}')

Количество пропущенных значений sqft после дополнения: 40049


In [27]:
# проверим содержание столбца
df['acres_from_beds'].unique()[0:500]

# изменим столбец на числовое значение и создадим новый столбец, в котором акры переведим в футы (1 акр = 43560 футов кв.)
df['acres_from_beds'] = df['acres_from_beds'].astype(float)
df['acres_from_beds_new'] = df['acres_from_beds'] * 43560

# заполним столбец данными из sqft_from_beds
df['sqft'] = df['sqft'].fillna(df['acres_from_beds_new'])

# смотрим снова количество пропусков
sqft_dataisnull_2 = df['sqft'].isnull().sum()
print(f'Количество пропущенных значений sqft после второго дополнения: {sqft_dataisnull_2}')

Количество пропущенных значений sqft после второго дополнения: 38458


In [28]:
# заменим пропуски на значение 0
df['sqft'] = df['sqft'].fillna(0)

# изменим тип данных
df['sqft'] = df['sqft'].astype(int)

# удалим лишние столбцы
df = df.drop(['sqft_from_beds', 'acres_from_beds', 'acres_from_beds_new'], axis=1)

Рассмотрим признак bath.

In [30]:
# выполним проверку
missing_values_count = df.baths.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.baths.nunique()}\n"
      f"Уникальные значения: {list(df.baths.unique()[:10])}")

Процент пропущенных значений: 28.19%
Количество уникальных значений: 229
Уникальные значения: ['3.5', '3 Baths', '2 Baths', '8 Baths', nan, '2', '3', 'Bathrooms: 2', '1,750', '4 Baths']


In [31]:
# Удалим все буквы, пробелы и двоеточия
df.baths = df.baths.str.replace('[a-zA-Z:]','', regex=True)
# Удаляем все пробелы
df.baths = df.baths.str.replace(' ','', regex=True)

Заполним пропуски нулями, так как большинство пропущенных значений означает отсутвие ванных комнат.

In [32]:
# исправим данные
df['baths'] = df['baths'].str.replace('[a-zA-Z+:]','', regex=True)
df['baths'].replace(['nan','', '~', '..', '--', '—','0.0','0.00','0/0'], 0, inplace=True)
df['baths'].replace(['1-2','1/1-0/1-0/1-0', '1/1/1/1','1-0/1-0/1', ], 1, inplace=True)
df['baths'].replace(['2-1/2-1/1-1/1-1','3-1/2-2',], 2, inplace=True)
df['baths'].replace(['116/116/116'], 116, inplace=True)
df['baths'].replace(['7,500'], 7.5, inplace=True)
df['baths'].replace(['5,000'], 5, inplace=True)
df['baths'].replace(['3,500'], 3.5, inplace=True)
df['baths'].replace(['2,750'], 2.75, inplace=True)
df['baths'].replace(['2,250'], 2.25, inplace=True)
df['baths'].replace(['1,250'], 1.25, inplace=True)
df['baths'].replace(['1,500'], 1.5, inplace=True)
df['baths'].replace(['4,000'], 4, inplace=True)
df['baths'].replace(['2,000'], 2, inplace=True)
df['baths'].replace(['3,000'], 3, inplace=True)
df['baths'].replace(['2,500'], 2.5, inplace=True)
df['baths'].replace(['1,000'], 1, inplace=True)
df['baths'].replace(['1,750'], 1.75, inplace=True)
df['baths'] = df['baths'].fillna(0)

In [33]:
# переведем в числовой признак и округлим до целого числа
df['baths'] = df['baths'].astype(float).round()

Поработаем с признаком city. Выявим пропуски и индексы без названия города.

In [38]:
# ищем пропуски
city_isnulldata = df['city'].isnull()

# ищем индексы без названия города
print(set(list(df[city_isnulldata]['zipcode'])))

{'34474', '38732', '34432', '34481', '33954', '78045', '34747', '32179', '34741', '32668', '77032', '34473', '32686', '34488', '20003', '34744', '33126', '33955'}


Создадим список городов по индексу и заполним пропуски по городам через индексы.

In [40]:
zipcode_city_list = {'20003':'Washington', '32179':'Ocklawaha', '32668':'Morriston', '32686':'Reddick', '33126':'Miami', '33954':'Port Charlotte',
                     '33955':'Punta Gorda', '34432':'Dunnellon','34473':'Ocala', '34474':'Ocala', '34481':'Ocala', '34488':'Silver Springs',
                     '34741':'KISSIMMEE', '34744':'Kissimmee', '34747':'Kissimmee	', '38732':'Cleveland', '77032':'Houston', '78045':'Laredo'}

def find_city (zipcode):
    # смотрим есть ли zipcode в словаре, и если есть, возвращаем соответствующее значение
    if zipcode in zipcode_city_list:
        return zipcode_city_list[zipcode]
    else:
        return None  # если zipcode не найден, возвращаем None

df['city_add'] = df['zipcode'].apply(find_city)

# применим функцию к столбцу 'zipcode' и записываем результаты в столбец 'city_add'
df['city_add'] = df['zipcode'].apply(find_city)

# заполним пропущенные данные в столбце city по столбцу city_add
df['city'].fillna(df['city_add'], inplace=True)

# проверим на пропуски
city_isnulldata_new = df['city'].isnull().mean()*100
print(f'Количество пропущенных значений столбец city: {city_isnulldata_new:.2f} %')

# удалим ненужный столбец
df = df.drop(['city_add'], axis=1)

Количество пропущенных значений столбец city: 0.00 %


Поработаем с признаком status.

In [44]:
missing_values_count = df.status.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.status.nunique()}\n"
      f"Уникальные значения: {list(df.status.unique()[:10])}")

Количество пропущенных значений: 39917
Процент пропущенных значений: 10.58%
Количество уникальных значений: 159
Уникальные значения: ['Active', 'for sale', nan, 'New construction', 'New', 'For sale', 'Pending', 'P', 'Active/Contingent', 'Pre-foreclosure / auction']


Разделим статусы на категории, сократив при этом количество уникальных значений и заменим пропуски на 'unknown'.

In [45]:
categories = {
    "For Sale": ["for sale", "For sale", "New construction", "New"],
    "Active": [
        "Active", "A Active", "Active/Contingent", "Active Under Contract", "Active Option", "Auction - Active",
        "Active With Contingencies", "Active Option Contract", "Active Contingency", "Active Backup",
        "Active Contingent", "Active - Auction", "Active With Offer", "Active - Contingent", "Active with Contract",
        "Temporary Active", "Re Activated", "Reactivated"
    ],
    "Pending": [
        "P", "Pending", "pending", "P Pending Sale", "Pending Ab", "Pending Continue To Show",
        "Pending Inspection", "Pending Offer Approval", "Pending In", "Pending W/Insp Finance", "Pending Fe",
        "Pending W/Backup Wanted", "Pending Backups Wanted", "Pending With Contingencies", "Lease/Purchase Pending",
        "Pending Bring Backup", "Pending - Taking Backups", "Pending - Continue to Show",
        "Pending Taking Backups", "Offer Pending Signature", "Pending (Do Not Show)", "Pending W/ Cont.",
        "Pending W/Escape Clause", "Pending - Backup Offer Requested", "Pending Sale"
    ],
    "Contingent": [
        "Contingent", "Contingent Finance And Inspection", "Contingent Show",
        "Contingent Take Backup", "Contingent - Sale of Home", "Contingent Finance and Inspection",
        "C Continue Show", "Contingent   Show", "Contingent   Release", "Contingent   No Show",
        "CT Insp - Inspection Contingency", "Contingent   Foreclosure", "Conting Accpt Backups",
        "Contingent - Financing", "Contingency 48 Hr (+/ )", "Contingency Contract", "Contingent Escape"
    ],
    "Foreclosure": [
        "Pre-foreclosure", "Pre-foreclosure / auction", " / auction", "Foreclosed", "foreclosure", "Foreclosure"
    ],
    "Under Contract": [
        "Under Contract", "Under Contract   Showing", "Under Contract Backups", "Under Contract Show",
        "Under Contract - Show", "Under Contract - No Show", "Under contract", "U Under Contract",
        "Due Diligence Period", "Contract Contingent On Buyer Sale", "Contract P", "Ct", "Uc Continue To Show",
        "Under Contract Taking Back Up Offers", "Under Contract W/ Bckp", "Contract"
    ],
    "For Rent": ["for rent", "Apartment for rent", "Condo for rent"],
    "Auction": ["Auction"],
    "Recently Sold": ["recently sold"],
    "Price Change": ["Price Change"],
    "Back on Market": ["Back on Market", "Back On Market"],
    "Closed": ["Closed"],
    "Listing Extended": ["Listing Extended"],
    "Coming Soon": [
        "Coming soon: Nov 21.", "Coming soon: Dec 4.", "Coming soon: Nov 23.", "Coming soon: Nov 29.",
        "Coming soon: Dec 2.", "Coming soon: Dec 10.", "Coming soon: Dec 24.", "Coming soon: Nov 14.",
        "Coming soon: Nov 22.", "Coming soon: Oct 21.", "Coming soon: Dec 14.", "Coming soon: Oct 24.",
        "Coming soon: Dec 18.", "Coming soon: Dec 16.", "Coming soon: Dec 3.", "Coming soon: Dec 25.",
        "Coming soon: Nov 11.", "Coming soon: Nov 28.", "Coming soon: Nov 17.", "Coming soon: Dec 6.",
        "Coming soon: Nov 27.", "Coming soon: Nov 26.", "Coming soon: Dec 7.", "Coming soon: Dec 27.",
        "Coming soon: Dec 11.", "Coming soon: Dec 5.", "Coming soon: Nov 13.", "Coming soon: Nov 19.",
        "Coming soon: Nov 8.", "Coming soon: Oct 29.", "Coming soon: Dec 15.", "Coming soon: Oct 30.",
        "Coming soon: Dec 9.", "Coming soon: Dec 20.", "Coming soon: Dec 13.", "Coming soon: Dec 23.",
        "Coming soon: Nov 30.", "Coming soon: Dec 1.", "Coming soon: Nov 5.", "Coming soon: Nov 12.",
        "Coming soon: Nov 25.", "Coming soon: Nov 9."
    ],
}

def categorize_status(status_value):
    for category, values in categories.items():
        if status_value in values:
            return category
    return "unknown"

# заменим значения на категории
df['status'] = df['status'].apply(categorize_status)

In [47]:
#снова проверим
missing_values_count = df.status.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.status.nunique()}\n"
      f"Уникальные значения: {list(df.status.unique()[:10])}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 15
Уникальные значения: ['Active', 'For Sale', 'unknown', 'Pending', 'Foreclosure', 'Under Contract', 'For Rent', 'Auction', 'Contingent', 'Price Change']


Рассмотрим признак propertyType.

In [48]:
missing_values_count = df.propertyType.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.propertyType.nunique()}\n"
      f"Уникальные значения: {list(df.propertyType.unique()[:10])}")

Количество пропущенных значений: 34733
Процент пропущенных значений: 9.21%
Количество уникальных значений: 1280
Уникальные значения: ['Single Family Home', 'single-family home', 'lot/land', 'townhouse', 'Florida', nan, 'Single Family', 'coop', 'English', '2 Story']


Получилось большое количество пропущенных значений. Объединим похожие значения распространенных типов, оставшиеся заменим на 'other'.

In [50]:
# переведем все типы к нижнему регистру
df['propertyType'] = df['propertyType'].str.lower()

# заменим похожие названия, выделила 9 основных типов, все остальные будут unknown
df['propertyType'] = df['propertyType'].replace(['single-family home', 'single family home', 'singlefamilyresidence', '1 story',
                                                'one story traditional', '1 story traditional', 'detached, one story',
                                                'single detached', 'single wide', 'single-wide mobile with land',
                                                '1 story, contemporary', '1 story, other (see remarks)',
                                                'single detached, french', '1 story, traditional', 'single detached, traditional',
                                                'one story', 'one level unit', '1 1/2 story', 'single wide mh',
                                                '1 story,traditional', '1 story, historic/older, traditional', '1 story, split level'],
                                        'single family')
df['propertyType'] = df['propertyType'].replace(['multi-family', 'multi-family home', 'duplex', 'triplex', 'fourplex',
                                                 'detached, two story', '2 story, other (see remarks)',
                                                 'multi_level', '2-story', 'two story', 'multi-level, modern', '2 stories, traditional',
                                                 '2 stories', 'traditional', 'attached or 1/2 duplex, traditional'],
                                        'multi family')
df['propertyType'] = df['propertyType'].replace(['coop', 'cooperative', 'condo/townhome/row home/co-op', 'condo/townhome','condominium',
                                                 'condo/unit', 'apartment/condo/townhouse', 'co-op', '2 story condo', 'high rise',
                                                 '2 unit condo', 'condo/townhome, contemporary/modern, loft, traditional',
                                                 'condo/townhome, hi-rise, resort property, vacation home, contemporary/modern',
                                                 'condo/townhome, other (see remarks)', 'condo/townhome, french',
                                                 'condo/townhome, hi-rise, contemporary/modern, loft', 'condo, other (see remarks)',
                                                 'condo/townhome, craftsman, traditional'],
                                        'condo')
df['propertyType'] = df['propertyType'].replace(['townhome style', 'townhouse-interior', 'townhouse-end unit',
                                                 'townhouse, attached/row', 'townhouse, northwestern contemporary', 'attached, townhouse',
                                                 'townhouse, 2-story', 'townhouse, two story, traditional', 'townhouse, villa'],
                                        'townhouse')
df['propertyType'] = df['propertyType'].replace(['lot/land'],
                                        'land')
df['propertyType'] = df['propertyType'].replace(['mobile/manufactured', 'mobile / manufactured', 'manufactured house', 'mfd/mobile home',
                                                 'manufactured home', 'manufactured double-wide', 'manufactured single-wide',
                                                 'mobile home 1 story', 'mobile manu - double wide','manufactured house,ranch, one story',
                                                 'manufactured house, ranch, one story, manufactured home',
                                                 'manufactured house, traditional, manufactured home', 'manufactured house, manufactured home',
                                                 '1 story, manufactured home - single wide', 'manufactured home, mobile home, ranch'],
                                        'mobile home')
df['propertyType'] = df['propertyType'].replace(['designated historical home', 'historical/conservation district', 'historic/older',
                                                 'historic vintage', 'historic', '1 story, historic/older, craftsman',
                                                 'historical/conservation district, single detached, contemporary/modern, traditional',
                                                 '1 story, historic/older, traditional, craftsman', 'historical, traditional',
                                                 '2 stories, historic/older, craftsman'],
                                        'historical')
df['propertyType'] = df['propertyType'].replace(['rancher', '1 story/ranch', '1 story, ranch', 'rancher, raised ranch', 'farms/ranches',
                                                 'hi ranch', 'ranch, one story', 'ranch, traditional', 'farm house',
                                                 'ranch, one story, duplex', 'farm house', 'ranch, transitional',
                                                 'bungalow, contemporary, ranch, traditional', '1 story, ranch, traditional, texas hill country',
                                                 '1 story, contemporary, ranch, historic/older, traditional', 'farm house, transitional',
                                                 'ranch, traditional, transitional', 'old world, ranch', 'ranch, split level', 
                                                 'ranch, spanish', 'farm/ranch house, single detached, contemporary/modern, ranch',
                                                 '2 stories, colonial, ranch', '1 story, ranch, craftsman', 'raised ranch, rancher'],
                                        'ranch')
df['propertyType'] = df['propertyType'].replace(['condominium (single level)', 'high-rise', 'mid-rise', 'low-rise (1-3 stories)',
                                                 'Flats', 'studio'],
                                        'apartment')
df['propertyType'] = df['propertyType'].replace('yes','other')

# заменим пропуски на other
df['propertyType'].fillna('other', inplace=True)

# распространненные типы
type_general = ['single family','multi family', 'condo', 'townhouse', 'land', 'mobile home', 'historical', 'ranch', 'apartment', 'unknown']
# функция проверяет, есть ли значение столбца в списке основных типов, если нет, то other
def change_type(type):
    if type in type_general:
        return type    # значение найдено в списке, остается без изменений
    else:
        return 'other'

# применим функцию
df['propertyType'] = df['propertyType'].apply(change_type)

# проверим уникальные значения и пропуски
uniq_propertyType_sum = df['propertyType'].nunique()
print(f'Количество уникальных значений столбца propertyType: {uniq_propertyType_sum}')

df['propertyType'].isnull().mean()*100

Количество уникальных значений столбца propertyType: 11


0.0

Проработаем признак street.

In [51]:
missing_values_count = df.street.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df.street.nunique()}")

Количество пропущенных значений: 2
Процент пропущенных значений: 0.00%
Количество уникальных значений: 337076


In [56]:
# удалим те строки, где есть пропуски в столбце street
df = df.dropna(subset=['street'])

Рассмотрим zipcode

In [58]:
missing_values_count = df['zipcode'].isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df['zipcode'].nunique()}\n"
f"Уникальные значения: {list(df['zipcode'].sort_values().unique()[:10])}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 4549
Уникальные значения: ['--', '0', '00000', '02108', '02109', '02110', '02111', '02113', '02114', '02115']


In [60]:
# выведем результат с некорректными значениями
df[(df['zipcode']=='--') | (df['zipcode']=='0')| (df['zipcode']=='00000') ]

,status,propertyType,street,baths,homeFacts,city,schools,sqft,zipcode,state,target,private_pool_final,beds_final
30258,Active,land,Gates Canyon Rd,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",Vacaville,"[{'rating': ['7', '4', '6', '6', '10', '2'], '...",0,0,CA,"$380,000",False,0.0
83517,For Sale,other,Cornejo Ricardo Descalzi,0.0,"{'atAGlanceFacts': [{'factValue': '1995', 'fac...",Quito Ecuador,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,0,NY,"470,000",False,0.0
231264,For Sale,other,0 N Gopher Canyon Rd,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",Bonsall,"[{'rating': ['7/10', '7/10', '3/10'], 'data': ...",19558440,--,CA,"$60,000,000",False,0.0
235188,unknown,townhouse,1744 N Dixie Hwy # 1744,3.0,"{'atAGlanceFacts': [{'factValue': '2010', 'fac...",Fort Lauderdale,"[{'rating': ['3/10', '5/10', '7/10'], 'data': ...",2043,--,FL,"$425,000",False,3.0
305540,For Sale,land,000 U.S. Hwy 359,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",Laredo,"[{'rating': ['4/10'], 'data': {'Distance': ['7...",243849,0,TX,"$1,740,000",False,0.0
308196,Active,other,435 pescador,0.0,"{'atAGlanceFacts': [{'factValue': '1972', 'fac...",Foreign Country,"[{'rating': [], 'data': {'Distance': [], 'Grad...",1100,00000,OS,"125,000",False,0.0


In [62]:
# отфильтруем индексы строк с некорректными значениями 
indexes_to_drop = df[(df['zipcode']=='--') | (df['zipcode']=='0')| (df['zipcode']=='00000')].index

# удалим строки с отфильтрованными индексами
df = df.drop(indexes_to_drop)

In [63]:
# смотри результат
df.zipcode.sort_values().unique()[:10]

array(['02108', '02109', '02110', '02111', '02113', '02114', '02115',
       '02116', '02118', '02119'], dtype=object)

In [65]:
# исправим не корректные zipcode, удалив вторую часть
df.zipcode = df.zipcode.str.replace("-.+",'', regex=True)

Обратим внимаение на признак state.

In [67]:
print(f"Количество пропущенных значений: {df.state.isna().sum()}\n"
      f"Количество уникальных значений: {df.state.nunique()}\n"
      f"Уникальные значения: {list(df.state.unique()[:10])}")

Количество пропущенных значений: 0
Количество уникальных значений: 38
Уникальные значения: ['NC', 'WA', 'CA', 'TX', 'FL', 'PA', 'TN', 'IA', 'NY', 'OR']


Пропусков нет, оставляем.

Посмотрим на признак schools.

In [68]:
df.schools[5]

"[{'rating': [], 'data': {'Distance': [], 'Grades': []}, 'name': []}]"

Как видим, признак состоит из рейтинга, расстояния до школы, оценки, названия. Последние два не представляют полезной информации, в отличие от рейтинга и расстояния до школы.

In [69]:
# рассчитаем средний рейтинг школы
def process_rating(rating):
    rating = rating[0].replace('/10', '')
    extracted_numbers = [float(num) for num in rating if num.isdigit()]
    average_rating = np.average(extracted_numbers) if extracted_numbers else -1
    return round(average_rating, 1)

s_rating = (
    df.schools.str.findall(r"\brating': ([\s\S]+?), 'data\b")
    .apply(lambda x: process_rating(x))
)

# создадим признак school_rating _mean в нашем наборе данных. Пропуски заполним -1 
df['school_rating _mean'] = s_rating

In [71]:
# расчитаем минимальную дистанцию
s_distance = df.schools.str.findall(r"\bDistance': ([\s\S]+?), 'Grades\b") # извлечение расстояний до школ из каждой строки
s_distance = s_distance.apply(lambda x: x[0]) # получение первого найденного значения из списка
s_distance = s_distance.str.replace('[a-zA-Z]','', regex=True) # удаление всех символов алфавита из расстояний
s_distance = s_distance.str.findall(r'\b([0-9]+.[0-9]+)') # извлечение чисел в каждой строке
s_distance = s_distance.apply(lambda x: [float(i) for i in x]) # преобразование в тип float
school_dist_min = s_distance.apply(lambda x: -1 if len(x)==0 else min(x)) # нахождение минимального значения в каждом списке s_distance, и возвращение -1, если длина списка равна 0

# создадим признак school_dist_min в нашем наборе данных. Пропуски заполним -1
df['school_dist_min'] = school_dist_min

# теперь можем удалить 
df.drop('schools', axis=1, inplace=True)

Поработаем с признаком homeFacts.

In [72]:
missing_values_count = df.homeFacts.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df.homeFacts.nunique()}\n"
f"Уникальные значения: {list(df.homeFacts.unique()[:1])}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 321004
Уникальные значения: ["{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': '$144', 'factLabel': 'Price/sqft'}]}"]


Как видим, признак состоит из информации о годе постройки дома, реставрации дома, кондиционере, отоплении, паркинге, площади и цене в футах. Рассмотрим данные.

In [73]:
# выберем значение Label и Value с помощью регулярных выражений
label = df.homeFacts.str.findall(r"\bfactLabel': ([\s\S]+?)[}\b]")
label[100]

["'Year built'",
 "'Remodeled year'",
 "'Heating'",
 "'Cooling'",
 "'Parking'",
 "'lotsize'",
 "'Price/sqft'"]

In [74]:
Value = df.homeFacts.str.findall(r"\bfactValue': ([\s\S]+?), 'factLabel\b")
Value[:10]

C:\Users\1\AppData\Local\Temp\ipykernel_9732\2695116100.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  Value[:10]


0    ['2019', '', 'Central A/C, Heat Pump', '', '',...
1    ['2019', '', '', '', '', '5828 sqft', '$159/sq...
2    ['1961', '1967', 'Forced Air', 'Central', 'Att...
3    ['2006', '2006', 'Forced Air', 'Central', 'Det...
4            ['', '', '', '', '', '10,019 sqft', None]
5    ['1920', '', 'Forced Air', 'Central', '', '680...
6    ['2006', '2006', 'Electric, Heat Pump', 'Centr...
7    ['1976', '', '', '', '', '8,750 Sq. Ft.', '$57...
8    ['1970', '', 'Forced Air', 'Central', '', '124...
9    ['2019', None, 'Gas', 'Central', 'Attached Gar...
Name: homeFacts, dtype: object

In [75]:
# создадим список новых признаков, удалив лишние кавычки
list_label = ','.join(label[0]).replace("'","").split(',')
list_label

['Year built',
 'Remodeled year',
 'Heating',
 'Cooling',
 'Parking',
 'lotsize',
 'Price/sqft']

In [76]:
# создадим эти признаки в нашем датафрейме и заполним значениями
for i, val in enumerate(list_label):
    df[val]=Value.apply(lambda x: x[i])

In [104]:
# теперь признак homeFacts можем удалить
df = df.drop('homeFacts', axis=1)

Теперь перейдем к обработке новых признаков.

Расссмотрим признак Year built.

In [77]:
#удалим лишние кавычки
df['Year built'] = df['Year built'].str.replace("'",'', regex=True)
#посмотрим на уникальные значения
df['Year built'].sort_values().unique()[:10]

array(['', '1', '1019', '1057', '1060', '1208', '1700', '1703', '1735',
       '1740'], dtype=object)

Годы постройки 1019', '1057', '1060 вероятно ошибочные и начинаются на 19..., поэтому исправим. А '', '1','1208', '2025', '559990649990', 'No Data' заменим на "unknown".

In [78]:
df['Year built'] = df['Year built'].str.replace('^\s*$','unknown', regex=True)
df['Year built'] = df['Year built'].str.replace('No Data','unknown')
df['Year built'] = df['Year built'].str.replace('559990649990','unknown')
df['Year built'] = df['Year built'].str.replace('^1$','unknown', regex=True)
df['Year built'] = df['Year built'].str.replace('None','unknown')

df['Year built'] = df['Year built'].str.replace('1060','1960')
df['Year built'] = df['Year built'].str.replace('1019','1919')
df['Year built'] = df['Year built'].str.replace('1057','1957')

Посмотрим теперь на признак Remodeled year.

In [79]:
#удалим лишние кавычки
df['Remodeled year'] = df['Remodeled year'].str.replace("'",'', regex=True)
#посмотрим на уникальные значения
df['Remodeled year'].sort_values().unique()[:10]

array(['', '0', '1111', '1738', '1800', '1845', '1846', '1853', '1862',
       '1869'], dtype=object)

In [80]:
# заменим некорректные значения на None
mask= df['Remodeled year'].isin(['', '0', '1111'])
df.loc[mask,'Remodeled year'] = 'None'
#Выведем уникальные значения
display(df['Remodeled year'].sort_values().unique()[:10])

array(['1738', '1800', '1845', '1846', '1853', '1862', '1869', '1870',
       '1874', '1876'], dtype=object)

In [81]:
# проверим на пропуски
missing_values_count = df['Remodeled year'].isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df['Remodeled year'].nunique()}\n"
f"Уникальные значения: {list(df['Remodeled year'].sort_values().unique()[:10])}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 152
Уникальные значения: ['1738', '1800', '1845', '1846', '1853', '1862', '1869', '1870', '1874', '1876']


In [82]:
# посмотрим на колличество пропуском в None в % соотношении
print(f"Процент None: {round(df[df['Remodeled year']=='None'].shape[0]/df.shape[0]*100, 2)}%")
print(f"Количество None: {df.loc[df['Remodeled year']=='None', 'Remodeled year'].count()}")

Процент None: 60.01%
Количество None: 226325


Обнаруживается слишком много пропусков, поэтому логично удалить этот признак.

In [83]:
df = df.drop('Remodeled year', axis=1)

Поработаем с признаком 'Heating'.

In [85]:
display(df['Heating'].str.lower().value_counts().head(20))

'forced air'                     134307
''                               105753
'other'                           29622
'electric'                        10216
'gas'                              9296
'heat pump'                        8851
'no data'                          8610
'central air'                      7814
'central electric'                 7112
'central'                          6247
'central, electric'                4253
'baseboard'                        3815
none                               3533
'wall'                             3301
'electric heat'                    3064
'heating system'                   2709
'forced air, heat pump'            1767
'radiant'                          1485
'central air, ceiling fan(s)'      1432
'natural gas heat'                 1383
Name: Heating, dtype: int64

Признак содержит информацию о различных типах отопления, но имеет большое колличество пропусков. Закодируем признаки булевыми переменными, где есть отопление - True (где есть записи), и где нет отопления('', 'No Data', 'No Heat Fuel', 'No Heat', 'None') - False.

In [86]:
# удалим лишние кавычки
df['Heating'] = df['Heating'].str.replace("'",'', regex=True)

# функция меняет столбец на булевые значения, там, где есть данные - True
df['Heating_final'] = df['Heating'].apply(lambda x: True if x not in ['', 'No Data', 'No Heat Fuel', 'No Heat', 'None'] else False)
df['Heating_final'].value_counts()

True     259154
False    117973
Name: Heating_final, dtype: int64

In [103]:
# удалим признак 'Heating'
df= df.drop('Heating', axis=1)

Проработаем признак Cooling.

In [88]:
df['Cooling'].str.lower().value_counts().head(20)

'central'                                            158743
''                                                   120390
'central air'                                         14384
'no data'                                             10615
'has cooling'                                          9730
'none'                                                 7390
'central electric'                                     6154
'wall'                                                 4017
'central gas'                                          3573
none                                                   3533
'central heating'                                      2807
'cooling system'                                       2700
'central a/c'                                          2051
'other'                                                1840
'central a/c (electric), central heat (gas)'           1646
'central a/c (electric), central heat (electric)'      1429
'refrigeration'                         

In [90]:
# удалим лишние кавычки
df['Cooling'] = df['Cooling'].str.replace("'",'', regex=True)
df['Cooling_encoded'] = df['Cooling'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none'] else False)

In [92]:
# проверим на пропуски в % соотношении
cooling_nodata_per = round(((df[df['Cooling']==''].shape[0]) / (df['Cooling'].shape[0]) * 100), 1)
print(f'Пропущенные данные в столбце Cooling: {cooling_nodata_per} %')

Пропущенные данные в столбце Cooling: 31.9 %


Признак Cooling содержит много пропусков, а также некорректной информации не только о наличии/отсутствии кондиционера, но еще о типах отопления, которые уже есть в признаке 'Heating'. Поэтому удаляем его.

In [93]:
# удалим признак 'Cooling'
df= df.drop('Cooling', axis=1)

Рассмотрим признак Parking.

In [95]:
df['Parking'].str.lower().value_counts().head(20)

''                                    171839
'attached garage'                      70748
'2 spaces'                             28061
'1 space'                              14252
'no data'                              13332
'detached garage'                      13200
'carport'                               7743
'off street'                            5279
'3 spaces'                              4724
none                                    3533
'carport, attached garage'              3025
'1'                                     2936
'4 spaces'                              2917
'2'                                     2756
'none'                                  2368
'on street'                             1707
'attached garage, detached garage'      1354
'0'                                     1114
'attached garage, carport'               993
'parking desc'                           900
Name: Parking, dtype: int64

In [97]:
#удалим лишние ковычки
df['Parking'] = df['Parking'].str.replace("'",'', regex=True)
df['Parking_encoded'] = df['Parking'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none','0'] else False)
# проверим на пропуски в % соотношении
Parking_nodata_per = round(((df[df['Parking']==''].shape[0]) / (df['Parking'].shape[0]) * 100), 1)
print(f'Пропущенные данные в столбце Parking: {Parking_nodata_per} %')

Пропущенные данные в столбце Parking: 45.6 %


Пропусков много, почти половина, поэтому удаляем признак.

In [98]:
df = df.drop('Parking', axis=1)

Признак lotsize показывает площадь территории, на которой находится дом, что не является информативным, поэтому удаляем его.

In [101]:
# удалим признак 'lotsize'
df = df.drop('lotsize', axis=1)

Признак Price/sqft напрямую зависит от целевого признака, что может привести к утечке данных. Удалим его.

In [102]:
# удалим признак 'Price/sqft'
df= df.drop('Price/sqft', axis=1)

**Займемся обработкой целевого признака target.**

In [106]:
# проверим на пропуски
missing_values_count = df['target'].isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {df['target'].nunique()}\n"
f"Уникальные значения: {list(df['target'].sort_values().unique()[:10])}")

Количество пропущенных значений: 2480
Процент пропущенных значений: 0.66%
Количество уникальных значений: 43938
Уникальные значения: ['$1', '$1,000', '$1,000,000', '$1,000,000+', '$1,000,036', '$1,000,050', '$1,000,100', '$1,000,898', '$1,000/mo', '$1,001,713']


Есть пропуски, поэтому удалим строки с пропусками, так как мы не можем их заполнить.

In [107]:
df = df.dropna(subset=['target'])

In [108]:
# удалим лишние символы
df['target'] = df['target'].replace('\$', '', regex=True)
df['target'] = df['target'].replace(',', '', regex=True)
df['target'] = df['target'].replace('\+', '', regex=True)

В признаке есть значения, которые показывают, что это цена за месяц с с пометкой /mo, рассмотрим их.

In [109]:
df[df['target'].str.contains('/mo',regex=True)].head()

,status,propertyType,street,baths,city,sqft,zipcode,state,target,private_pool_final,beds_final,school_rating _mean,school_dist_min,Year built,Heating_final,Cooling_encoded,Parking_encoded
547,For Rent,single family,4323 N Central Park Ave,4.0,Chicago,3300,60618,IL,5500/mo,False,4.0,2.3,0.13,1913,True,False,True
609,For Rent,multi family,220 Boylston St #1412,2.0,Boston,1673,2116,MA,10500/mo,False,2.0,-1.0,-1.00,1985,False,False,False
2075,For Rent,single family,2830 NE 56th Ct,4.0,Fort Lauderdale,2400,33308,FL,6390/mo,True,4.0,4.0,1.19,1965,False,False,False
3025,For Rent,multi family,411 Kline Aly,2.0,Clarksville,1280,37040,TN,1200/mo,False,2.0,8.0,0.68,2014,True,False,False
3645,For Rent,multi family,240 E Illinois St #2011,2.0,Chicago,1473,60611,IL,3600/mo,True,2.0,5.5,0.72,2003,False,False,True


Поскольку задача спрогнозировать стоимость недвижимости, а тут мы видим "For Rent" - стоимость аренды, удалим такие строки.

In [110]:
df = df[~df['target'].str.contains('/mo', regex=True)]

In [111]:
# переведем столбец в числовой формат
df['target'] = df['target'].astype(int)

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374249 entries, 0 to 377134
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               374249 non-null  object 
 1   propertyType         374249 non-null  object 
 2   street               374249 non-null  object 
 3   baths                374249 non-null  float64
 4   city                 374249 non-null  object 
 5   sqft                 374249 non-null  int32  
 6   zipcode              374249 non-null  object 
 7   state                374249 non-null  object 
 8   target               374249 non-null  int32  
 9   private_pool_final   374249 non-null  bool   
 10  beds_final           374249 non-null  float64
 11  school_rating _mean  374249 non-null  float64
 12  school_dist_min      374249 non-null  float64
 13  Year built           374249 non-null  object 
 14  Heating_final        374249 non-null  bool   
 15  Cooling_encoded  

In [113]:
# сохраняем наши предобработанные данные в CSV-файл для дальнейшей работы
df.to_csv("data/cleaned_data.csv", index=False)